# A New Tree Booster: PART

__12 Feb 2018, marugari__

PART (Peeking Additive Regression Trees) aims to
* optimize non-differential metrics
* avoid over-fitting

For training a PART boosters, we need split training data into 3 part.
1. training set: to search optimal splits
2. peeking set: to determin whether a new tree is committed
3. validation set: to get validation score

[Repository (https://github.com/marugari/LightGBM/tree/part)](https://github.com/marugari/LightGBM/tree/part)

This is implemented as a LightGBM custom booster.
The following is a fork of [the Kaggle Zillow Prize Kernel](https://www.kaggle.com/guolinke/simple-lightgbm-starter-lb-0-06487/code).

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

In [2]:
train = pd.read_csv('input/zillow/train_2016_v2.csv', engine='python')
prop = pd.read_csv('input/zillow/properties_2016.csv', engine='python')

In [3]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)
df_train = train.merge(prop, how='left', on='parcelid')
col = [
    'parcelid',
    'logerror',
    'transactiondate',
    'propertyzoningdesc',
    'propertycountylandusecode'
]
x_train = df_train.drop(col, axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)
train_columns = x_train.columns
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)
del df_train

(90275, 55) (90275,)


In [4]:
split = 80000
xt, xv = x_train[:split], x_train[split:]
xt = xt.values.astype(np.float32, copy=False)
xv = xv.values.astype(np.float32, copy=False)
yt, yv = y_train[:split], y_train[split:]
ds_train = lgb.Dataset(xt, label=yt, free_raw_data=False)
ds_valid = lgb.Dataset(xv, label=yv, free_raw_data=False)

In [5]:
prm = {
    'learning_rate': 0.002,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'sub_feature': 0.5,
    'num_leaves': 60,
    'min_data': 500,
    'min_hessian': 1,
}
num_round = 500

In [6]:
clf_gbdt = lgb.train(prm, ds_train, num_round, valid_sets=ds_valid)

[1]	valid_0's l1: 0.066548
[2]	valid_0's l1: 0.0665464
[3]	valid_0's l1: 0.0665447
[4]	valid_0's l1: 0.0665415
[5]	valid_0's l1: 0.0665396
[6]	valid_0's l1: 0.066537
[7]	valid_0's l1: 0.066535
[8]	valid_0's l1: 0.0665326
[9]	valid_0's l1: 0.0665298
[10]	valid_0's l1: 0.0665273
[11]	valid_0's l1: 0.0665251
[12]	valid_0's l1: 0.0665231
[13]	valid_0's l1: 0.0665205
[14]	valid_0's l1: 0.0665188
[15]	valid_0's l1: 0.0665166
[16]	valid_0's l1: 0.0665147
[17]	valid_0's l1: 0.0665127
[18]	valid_0's l1: 0.0665106
[19]	valid_0's l1: 0.066509
[20]	valid_0's l1: 0.0665068
[21]	valid_0's l1: 0.0665037
[22]	valid_0's l1: 0.0665015
[23]	valid_0's l1: 0.0664988
[24]	valid_0's l1: 0.066497
[25]	valid_0's l1: 0.0664948
[26]	valid_0's l1: 0.0664922
[27]	valid_0's l1: 0.0664895
[28]	valid_0's l1: 0.0664873
[29]	valid_0's l1: 0.0664852
[30]	valid_0's l1: 0.066484
[31]	valid_0's l1: 0.0664824
[32]	valid_0's l1: 0.0664797
[33]	valid_0's l1: 0.0664781
[34]	valid_0's l1: 0.0664758
[35]	valid_0's l1: 0.0664731


[280]	valid_0's l1: 0.0661595
[281]	valid_0's l1: 0.066159
[282]	valid_0's l1: 0.0661583
[283]	valid_0's l1: 0.0661574
[284]	valid_0's l1: 0.0661571
[285]	valid_0's l1: 0.0661568
[286]	valid_0's l1: 0.066156
[287]	valid_0's l1: 0.0661556
[288]	valid_0's l1: 0.0661549
[289]	valid_0's l1: 0.0661544
[290]	valid_0's l1: 0.0661544
[291]	valid_0's l1: 0.0661536
[292]	valid_0's l1: 0.066153
[293]	valid_0's l1: 0.0661525
[294]	valid_0's l1: 0.0661518
[295]	valid_0's l1: 0.0661512
[296]	valid_0's l1: 0.0661507
[297]	valid_0's l1: 0.0661502
[298]	valid_0's l1: 0.06615
[299]	valid_0's l1: 0.0661498
[300]	valid_0's l1: 0.0661493
[301]	valid_0's l1: 0.0661487
[302]	valid_0's l1: 0.0661487
[303]	valid_0's l1: 0.066148
[304]	valid_0's l1: 0.0661477
[305]	valid_0's l1: 0.0661477
[306]	valid_0's l1: 0.0661472
[307]	valid_0's l1: 0.0661466
[308]	valid_0's l1: 0.0661462
[309]	valid_0's l1: 0.066146
[310]	valid_0's l1: 0.0661456
[311]	valid_0's l1: 0.066145
[312]	valid_0's l1: 0.0661441
[313]	valid_0's l1

In [7]:
prm_part = prm
prm_part['boosting_type'] = 'part'
prm_part['learning_rate'] = 0.002
prm_part['drop_rate'] = 0.0
prm_part['skip_drop'] = 0.0
np.random.seed(20180212)
flg_part = np.random.choice([True, False], len(yt), replace=True, p=[0.7, 0.3])
flg_peek = np.logical_not(flg_part)
ds_part = lgb.Dataset(xt[flg_part], label=yt[flg_part], free_raw_data=False)
ds_peek = lgb.Dataset(xt[flg_peek], label=yt[flg_peek], free_raw_data=False)

In [8]:
clf_part = lgb.train(prm_part, ds_part, num_round, valid_sets=ds_peek)

[1]	valid_0's l1: 0.0683414
[2]	valid_0's l1: 0.0683379
[3]	valid_0's l1: 0.0683343
[4]	valid_0's l1: 0.068331
[5]	valid_0's l1: 0.0683291
[6]	valid_0's l1: 0.0683264
[7]	valid_0's l1: 0.0683249
[8]	valid_0's l1: 0.0683225
[9]	valid_0's l1: 0.06832
[10]	valid_0's l1: 0.0683163
[11]	valid_0's l1: 0.0683139
[12]	valid_0's l1: 0.0683106
[13]	valid_0's l1: 0.0683076
[14]	valid_0's l1: 0.0683049
[15]	valid_0's l1: 0.0683014
[16]	valid_0's l1: 0.0682984
[17]	valid_0's l1: 0.0682964
[18]	valid_0's l1: 0.0682937
[19]	valid_0's l1: 0.0682904
[20]	valid_0's l1: 0.0682873
[21]	valid_0's l1: 0.0682854
[22]	valid_0's l1: 0.0682819
[23]	valid_0's l1: 0.0682799
[24]	valid_0's l1: 0.068277
[25]	valid_0's l1: 0.0682755
[26]	valid_0's l1: 0.0682727
[27]	valid_0's l1: 0.0682709
[28]	valid_0's l1: 0.0682689
[29]	valid_0's l1: 0.068267
[30]	valid_0's l1: 0.0682641
[31]	valid_0's l1: 0.0682614
[32]	valid_0's l1: 0.0682589
[33]	valid_0's l1: 0.0682562
[34]	valid_0's l1: 0.0682533
[35]	valid_0's l1: 0.0682514

[280]	valid_0's l1: 0.0679006
[281]	valid_0's l1: 0.0679003
[282]	valid_0's l1: 0.067899
[283]	valid_0's l1: 0.0678986
[284]	valid_0's l1: 0.0678976
[285]	valid_0's l1: 0.0678962
[286]	valid_0's l1: 0.0678956
[287]	valid_0's l1: 0.0678946
[288]	valid_0's l1: 0.0678936
[289]	valid_0's l1: 0.0678932
[290]	valid_0's l1: 0.067892
[291]	valid_0's l1: 0.067891
[292]	valid_0's l1: 0.0678904
[293]	valid_0's l1: 0.0678898
[294]	valid_0's l1: 0.0678894
[295]	valid_0's l1: 0.0678881
[296]	valid_0's l1: 0.0678875
[297]	valid_0's l1: 0.0678871
[298]	valid_0's l1: 0.0678866
[299]	valid_0's l1: 0.0678863
[300]	valid_0's l1: 0.0678855
[301]	valid_0's l1: 0.0678842
[302]	valid_0's l1: 0.0678832
[303]	valid_0's l1: 0.0678825
[304]	valid_0's l1: 0.0678819
[305]	valid_0's l1: 0.0678813
[306]	valid_0's l1: 0.0678804
[307]	valid_0's l1: 0.0678798
[308]	valid_0's l1: 0.0678791
[309]	valid_0's l1: 0.0678784
[310]	valid_0's l1: 0.0678776
[311]	valid_0's l1: 0.0678769
[312]	valid_0's l1: 0.0678758
[313]	valid_0

In [9]:
from sklearn.metrics import mean_absolute_error
def get_score(x, y, clf, ii):
    return mean_absolute_error(y, clf.predict(x, num_iteration=ii))
lab = []
val_gbdt = []
val_part = []
ii = 25
while ii <= num_round:
    lab.append(ii)
    val_gbdt.append(get_score(xv, yv, clf_gbdt, ii))
    val_part.append(get_score(xv, yv, clf_part, ii))
    ii += 25

In [10]:
import plotly
plotly.offline.init_notebook_mode(connected=True)
dat = [
    plotly.graph_objs.Scatter(
        x=lab,
        y=val_gbdt,
        name='GBDT'
    ),
    plotly.graph_objs.Scatter(
        x=lab,
        y=val_part,
        name='PART'
    ),
]
figure = {
    'data': dat,
    'layout': {},
}
plotly.offline.iplot(
    figure,
    config={
        'displayModeBar': False,
        'showLink': False,
    }
)